In [1]:
!pip install datasets accelerate hf-transfer


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install evaluate


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset
# Load SST-2 dataset
dataset = load_dataset("glue", "qqp")

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-large")

teacher_cls = AutoModelForSequenceClassification.from_pretrained("answerdotai/ModernBERT-large",  num_labels=2, problem_type="single_label_classification")
student_cls = AutoModelForSequenceClassification.from_pretrained("kartikeya-pandey/MiniModernBERT-Pretrained", num_labels=2,problem_type="single_label_classification")

teacher_cls = teacher_cls.to("cuda")
student_cls = student_cls.to("cuda")

c:\Users\Kartikeya\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at kartikeya-pandey/MiniModernBERT-Pretrained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def preprocess(example):
    return tokenizer(example["question1"], example["question2"], truncation=True, padding="max_length", max_length=128)

In [6]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [7]:
import torch
import torch.nn.functional as F
from torch.nn import KLDivLoss, CrossEntropyLoss

def compute_distill_loss(student_logits, teacher_logits, labels, alpha=0.6, temperature=1.0):
    # KL divergence between teacher and student (softened logits)
    kl_loss = KLDivLoss(reduction="batchmean")(
        F.log_softmax(student_logits / temperature, dim=-1),
        F.softmax(teacher_logits / temperature, dim=-1)
    ) * (temperature ** 2)

    # Cross entropy for classification
    ce_loss = CrossEntropyLoss()(student_logits, labels)

    return alpha * kl_loss + (1 - alpha) * ce_loss

In [8]:
# Custom compute_loss for Trainer
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs["labels"].long()
    with torch.no_grad():
        teacher_outputs = teacher_cls(**inputs)

    student_outputs = model(**inputs)
    loss = compute_distill_loss(
        student_outputs.logits,
        teacher_outputs.logits,
        labels
    )

    return (loss, student_outputs) if return_outputs else loss

In [9]:
student_cls.compute_loss = compute_loss.__get__(student_cls)

In [10]:
from huggingface_hub import notebook_login

# This will prompt you to enter your token securely in the notebook
notebook_login()

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

FileNotFoundError: Couldn't find a module script at d:\RM\research paper\mini-morder-bert-code\mini-mordern-bert\accuracy\accuracy.py. Module 'accuracy' doesn't exist on the Hugging Face Hub either.

In [ ]:
from transformers import TrainingArguments, Trainer

# Training arguments
args = TrainingArguments(
    output_dir=".\MiniModernBERT-glue-qqp",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=500,
    report_to="none",
    push_to_hub=True,
    hub_model_id="kartikeyapandey20/MiniModernBERT-qqp",
)



def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels)["f1"]
    }

trainer = Trainer(
    model=student_cls,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -------------------
# 9. Train & Evaluate
# -------------------
trainer.train()


FileNotFoundError: Couldn't find a module script at d:\RM\research paper\mini-morder-bert-code\mini-mordern-bert\accuracy\accuracy.py. Module 'accuracy' doesn't exist on the Hugging Face Hub either.

In [ ]:
# Remove the custom loss before normal evaluation
del student_cls.compute_loss

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6045704483985901,
 'eval_runtime': 1.637,
 'eval_samples_per_second': 532.673,
 'eval_steps_per_second': 66.584,
 'epoch': 2.0}